In [39]:
import numpy as np
import datetime as dt
import pandas as pd
import pyspedas
from pytplot import del_data, get_data, get_timespan, store_data, tplot_options, tplot_names, tplot, tplot_math
import matplotlib.pyplot as plt

In [40]:
def get_omni(start, stop, fixed):
    if fixed:
        start = start+dt.timedelta(minutes=30)

    trange = [start.strftime("%Y-%m-%d/%H:%M:%S"), stop.strftime("%Y-%m-%d/%H:%M:%S")] #Timeseries should be in the format [pd.Timestamp, pd.Timestamp]
    omni_import = pyspedas.omni.data(trange=trange, datatype='1min', level='hro2', time_clip=True)

    omni = {'Time': get_data('IMF')[0], 'BX_GSE': get_data('BX_GSE')[1], 'BY_GSE': get_data('BY_GSE')[1], 'BZ_GSE': get_data('BZ_GSE')[1], 'Vx': get_data('Vx')[1], 'Vy': get_data('Vy')[1], 'Vz': get_data('Vz')[1], 'proton_density': get_data('proton_density')[1], 'T': get_data('T')[1]}
    omni_data = pd.DataFrame(omni, columns=['Time', 'BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']).replace(to_replace=[999.990, 9999.99, 99999.9], value=np.nan)
    omni_data['Time'] = pd.to_datetime(omni_data['Time'], unit='s')

    omni_data = omni_data.set_index('Time').interpolate(method='linear')

    return omni_data

In [41]:
def get_artemis(start, stop, fixed):
    if fixed:
        start = start+dt.timedelta(minutes=30)

    trange = [start.strftime("%Y-%m-%d/%H:%M:%S"), stop.strftime("%Y-%m-%d/%H:%M:%S")] #Timeseries should be in the format [pd.Timestamp, pd.Timestamp]
    artemis_fgm_import = pyspedas.themis.fgm(trange=trange, probe='b', time_clip=True, varnames='thb_fgs_gse')
    artemis_esa_import = pyspedas.themis.esa(trange=trange, probe='b', time_clip=True, varnames=['thb_peif_velocity_gse', 'thb_peif_density', 'thb_peif_avgtemp'])

    #---ARTEMIS FGM data manipulation-------------------------------------------------------------------------------------------------------
    fgm = {'Time': get_data('thb_fgs_gse')[0], 'BX_GSE': get_data('thb_fgs_gse')[1][:,0], 'BY_GSE': get_data('thb_fgs_gse')[1][:,1], 'BZ_GSE': get_data('thb_fgs_gse')[1][:,2]}
    artemis_fgm_data = pd.DataFrame(fgm, columns=['Time', 'BX_GSE', 'BY_GSE', 'BZ_GSE']).replace(to_replace=[999.990, 9999.99, 99999.9], value=np.nan)
    artemis_fgm_data['Time'] = pd.to_datetime(artemis_fgm_data['Time'], unit='s')

    artemis_fgm_data = artemis_fgm_data.set_index('Time').resample('T').mean().interpolate(method='linear')
    #--------------------------------------------------------------------------------------------------------------------------------------

    #---ARTEMIS ESA data manipulation-------------------------------------------------------------------------------------------------------
    esa = {'Time': get_data('thb_peif_velocity_gse')[0], 'Vx': get_data('thb_peif_velocity_gse')[1][:,0], 'Vy': get_data('thb_peif_velocity_gse')[1][:,1], 'Vz': get_data('thb_peif_velocity_gse')[1][:,2], 'proton_density': get_data('thb_peif_density')[1], 'T': get_data('thb_peif_avgtemp')[1]}
    artemis_esa_data = pd.DataFrame(esa, columns=['Time', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']).replace(to_replace=[999.990, 9999.99, 99999.9], value=np.nan)
    artemis_esa_data['Time'] = pd.to_datetime(artemis_esa_data['Time'], unit='s')

    artemis_esa_data = artemis_esa_data.set_index('Time').resample('T').mean().interpolate(method='linear')
    artemis_esa_data = artemis_esa_data.reindex(artemis_fgm_data.index)
    #---------------------------------------------------------------------------------------------------------------------------------------

    return artemis_fgm_data, artemis_esa_data

In [42]:
def get_themis(start, stop, probe, fixed):
    if fixed:
        start = start+dt.timedelta(minutes=30)

    trange = [start.strftime("%Y-%m-%d/%H:%M:%S"), stop.strftime("%Y-%m-%d/%H:%M:%S")] #Timeseries should be in the format [pd.Timestamp, pd.Timestamp]
    themis_fgm_import = pyspedas.themis.fgm(trange=trange, probe=probe, time_clip=True, varnames='th'+probe+'_fgs_gse')
    themis_esa_import = pyspedas.themis.esa(trange=trange, probe=probe, time_clip=True, varnames=['th'+probe+'_peif_velocity_gse', 'th'+probe+'_peif_density', 'th'+probe+'_peif_avgtemp'])

    #---THEMIS FGM data manipulation-------------------------------------------------------------------------------------------------------
    fgm = {'Time': get_data('th'+probe+'_fgs_gse')[0], 'BX_GSE': get_data('th'+probe+'_fgs_gse')[1][:,0], 'BY_GSE': get_data('th'+probe+'_fgs_gse')[1][:,1], 'BZ_GSE': get_data('th'+probe+'_fgs_gse')[1][:,2]}
    themis_fgm_data = pd.DataFrame(fgm, columns=['Time', 'BX_GSE', 'BY_GSE', 'BZ_GSE']).replace(to_replace=[999.990, 9999.99, 99999.9], value=np.nan)
    themis_fgm_data['Time'] = pd.to_datetime(themis_fgm_data['Time'], unit='s')

    themis_fgm_data = themis_fgm_data.set_index('Time').resample('T').mean().interpolate(method='linear')
    #--------------------------------------------------------------------------------------------------------------------------------------

    #---THEMIS ESA data manipulation-------------------------------------------------------------------------------------------------------
    esa = {'Time': get_data('th'+probe+'_peif_velocity_gse')[0], 'Vx': get_data('th'+probe+'_peif_velocity_gse')[1][:,0], 'Vy': get_data('th'+probe+'_peif_velocity_gse')[1][:,1], 'Vz': get_data('th'+probe+'_peif_velocity_gse')[1][:,2], 'proton_density': get_data('th'+probe+'_peif_density')[1], 'T': get_data('th'+probe+'_peif_avgtemp')[1]}
    themis_esa_data = pd.DataFrame(esa, columns=['Time', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']).replace(to_replace=[999.990, 9999.99, 99999.9], value=np.nan)
    themis_esa_data['Time'] = pd.to_datetime(themis_esa_data['Time'], unit='s')

    themis_esa_data = themis_esa_data.set_index('Time').resample('T').mean().interpolate(method='linear')
    themis_esa_data = themis_esa_data.reindex(themis_fgm_data.index)
    #--------------------------------------------------------------------------------------------------------------------------------------

    return themis_fgm_data, themis_esa_data

In [114]:
def correlate(sliding, fixed):

    rows = []
    for j in range(len(fixed['Time'])-61): # Take the number of data points and subtract an hour from it. That's how many times you can iterate a one-hour chunk
        sliding_start = (sliding.loc[sliding['Time'] == fixed['Time'][j]]).index[0] # Set the start time of the sliding index where the timestamp of the fixed series begins
        sliding_stop = (sliding.loc[sliding['Time'] == fixed['Time'][j+60]]).index[0] # Set the end of the sliding index an hour after the start of the fixed series


        keys = ['BX_GSE', 'BY_GSE', 'BZ_GSE', 'Vx', 'Vy', 'Vz', 'proton_density', 'T']
        maxes = []
        offsets = []
        averages = []

        for k in keys:
            store = []
            for n in range(20): # Calculate a coefficient over a 30 min window
                coef = np.corrcoef(fixed[k][j:j+60], sliding[k][sliding_start-n:sliding_stop-n])[0, 1]
                if coef < 0:
                    coef = 0
                store.append(coef) # Append the storage arrays

            maxes.append(max(store))
            offsets.append(store.index(max(store)))
            averages.append(np.average(fixed[k][j:j+60]))

        rows.append(np.concatenate(([fixed['Time'][j]], [fixed['Time'][j+60]], maxes, offsets, averages), axis=None))

    return rows # Output the entire chunk of metadata

In [123]:
def import_events(path, satellite1, satellite2): # Function to import the data from a satellite to compare with Artemis: either OMNI, THEMIS, or MMS
    events = pd.read_csv(path, delimiter=',', header=0)
    events['time_start'] = pd.to_datetime(events['time_start'])
    events['time_stop'] = pd.to_datetime(events['time_stop'])

    for event in range(len(events['time_start'])):

        if satellite1 == 'artemis':
            artemis_fgm, artemis_esa = get_artemis(events['time_start'][event], events['time_stop'][event], fixed=False)
            sat1 = pd.concat([artemis_fgm, artemis_esa], axis=1).interpolate(method='linear').fillna(method='bfill')
            sat1_name = 'Artemis'

        elif satellite1 == 'themis' and satellite2 != 'themis':
            themis_fgm, themis_esa = get_themis(events['time_start'][event], events['time_stop'][event], probe=events['THEMIS'][event], fixed=False)
            sat1 = pd.concat([themis_fgm, themis_esa], axis=1).interpolate(method='linear').fillna(method='bfill')
            sat1_name = 'Themis'

        else:
            raise ValueError("String '"+satellite1+"' is not one of ('artemis', 'themis')")

        if satellite2 == 'themis' and satellite1 != 'themis':
            themis_fgm, themis_esa = get_themis(events['time_start'][event], events['time_stop'][event], probe=events['THEMIS'][event], fixed=True)
            sat2 = pd.concat([themis_fgm, themis_esa], axis=1).interpolate(method='linear').fillna(method='bfill')
            sat2_name = 'Themis'
        elif satellite2 == 'omni':
            sat2 = get_omni(events['time_start'][event], events['time_stop'][event], fixed=True)
            sat2_name = 'Omni'
        else:
            raise ValueError("String '"+satellite2+"' is not one of ('themis', 'omni')")


        sat1.reset_index(inplace=True)
        sat2.reset_index(inplace=True)

        a = pd.DataFrame(correlate(sat1, sat2), columns=['Start', 'Stop', 'P(Bx)', 'P(By)', 'P(Bz)', 'P(Vx)', 'P(Vy)', 'P(Vz)', 'P(n)', 'P(T)', 'Offset (Bx)', 'Offset (By)', 'Offset (Bz)', 'Offset (Vx)', 'Offset (Vy)', 'Offset (Vz)', 'Offset (n)', 'Offset (T)', 'Avg Bx', 'Avg By', 'Avg Bz', 'Avg Vx', 'Avg Vy', 'Avg Vz', 'Avg n', 'Avg T'])

        sat1.to_csv('../metadata/'+sat1_name+'/'+sat1_name+'_'+events['time_start'][event].strftime("%Y-%m-%d_%H-%M")+'.csv')
        sat2.to_csv('../metadata/'+sat2_name+'/'+sat2_name+'_'+events['time_start'][event].strftime("%Y-%m-%d_%H-%M")+'.csv')

        a.to_csv('../metadata/'+sat1_name+'-'+sat2_name+'_'+events['time_start'][event].strftime("%Y-%m-%d_%H-%M")+'.csv')


In [ ]:
import_events('../eventlist/eventlist.csv', 'artemis', 'omni')

08-Mar-23 19:00:24: Downloading remote index: http://themis.ssl.berkeley.edu/data/themis/thb/l2/fgm/2011/
08-Mar-23 19:00:24: File is current: themis_data/thb/l2/fgm/2011/thb_l2_fgm_20111024_v01.cdf
08-Mar-23 19:00:24: Time clip was applied to: thb_fgs_gse
08-Mar-23 19:00:24: Downloading remote index: http://themis.ssl.berkeley.edu/data/themis/thb/l2/esa/2011/
08-Mar-23 19:00:25: File is current: themis_data/thb/l2/esa/2011/thb_l2_esa_20111024_v01.cdf
08-Mar-23 19:00:25: Time clip was applied to: thb_peif_velocity_gse
08-Mar-23 19:00:25: Time clip was applied to: thb_peif_density
08-Mar-23 19:00:25: Time clip was applied to: thb_peif_avgtemp
08-Mar-23 19:00:25: Downloading remote index: https://spdf.gsfc.nasa.gov/pub/data/omni/omni_cdaweb/hro2_1min/2011/
08-Mar-23 19:00:25: File is current: omni_data/hro2_1min/2011/omni_hro2_1min_20111001_v01.cdf
08-Mar-23 19:00:26: Time clip was applied to: IMF
08-Mar-23 19:00:26: Time clip was applied to: PLS
08-Mar-23 19:00:26: Time clip was applied